### Danger Level Forecasting Preliminary Models

Testing out some of the classification models

#### Import Tools

In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

#### Import Data

In [9]:
avi = pd.read_csv('SnowWeatherClean.csv')
avi.head()

,Unnamed: 0,avi_danger,avg_wind,temp_max_swing,temp_max_swing_from_avg,year,month,day,temp_max,temp_min,...,northwest_wind_hours,prevailing_wind_E,prevailing_wind_N,prevailing_wind_NE,prevailing_wind_NW,prevailing_wind_S,prevailing_wind_SE,prevailing_wind_SW,prevailing_wind_W,next_day_avi_danger
0,0,2.0,10.74,0.0,0.0,2010.0,12.0,18.0,12,4,...,12,0,0,0,1,0,0,0,0,1.0
1,1,1.0,9.40,3.0,0.0,2010.0,12.0,19.0,15,3,...,12,0,0,0,1,0,0,0,0,1.0
2,2,1.0,20.58,0.0,0.0,2010.0,12.0,20.0,15,5,...,0,0,1,0,0,0,0,0,0,3.0
3,3,3.0,35.12,3.0,0.0,2010.0,12.0,21.0,18,10,...,0,0,1,0,0,0,0,0,0,2.0
4,4,2.0,33.78,-3.0,0.0,2010.0,12.0,22.0,15,7,...,1,0,1,0,0,0,0,0,0,3.0


#### filter data

In [11]:
avi = avi.drop(['Unnamed: 0'], axis=1)

In [13]:
avi = avi[avi['avi_danger'].notnull()]
avi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1253 entries, 0 to 1252
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   avi_danger               1253 non-null   float64
 1   avg_wind                 1253 non-null   float64
 2   temp_max_swing           1253 non-null   float64
 3   temp_max_swing_from_avg  1253 non-null   float64
 4   year                     1253 non-null   float64
 5   month                    1253 non-null   float64
 6   day                      1253 non-null   float64
 7   temp_max                 1253 non-null   int64  
 8   temp_min                 1253 non-null   int64  
 9   water_equivalent         1253 non-null   float64
 10  snow_fall                1253 non-null   float64
 11  snow_depth_6am           1253 non-null   float64
 12  wind_speed_sum           1253 non-null   int64  
 13  sunshine_percent         1253 non-null   int64  
 14  west_wind_hours         

#### Fill Remaing NA's

In [14]:
avi = avi.fillna(0)
avi = avi[avi.avi_danger != 5]
avi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1251 entries, 0 to 1252
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   avi_danger               1251 non-null   float64
 1   avg_wind                 1251 non-null   float64
 2   temp_max_swing           1251 non-null   float64
 3   temp_max_swing_from_avg  1251 non-null   float64
 4   year                     1251 non-null   float64
 5   month                    1251 non-null   float64
 6   day                      1251 non-null   float64
 7   temp_max                 1251 non-null   int64  
 8   temp_min                 1251 non-null   int64  
 9   water_equivalent         1251 non-null   float64
 10  snow_fall                1251 non-null   float64
 11  snow_depth_6am           1251 non-null   float64
 12  wind_speed_sum           1251 non-null   int64  
 13  sunshine_percent         1251 non-null   int64  
 14  west_wind_hours         

#### Splitting Dataset

In [15]:
x = avi.iloc[:, 1:avi.shape[1]]
y = avi.iloc[:, 0]

print(x.shape)
print(y.shape)

(1251, 24)
(1251,)


#### Standardizing

In [16]:
#standardizer 
def standardize(X_train, X_test):
    scaler = StandardScaler()
    # Fitting and transforming training data
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    # Tranforming testing data based on traning fit (prevent data leakage)
    X_test = scaler.transform(X_test)
    return X_train, X_test

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

# Scaling
x_train, x_test = standardize(x_train, x_test)

In [18]:
print(y)

(unique, counts) = np.unique(y, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)


0       2.0
1       1.0
2       1.0
3       3.0
4       2.0
       ... 
1248    1.0
1249    3.0
1250    3.0
1251    3.0
1252    2.0
Name: avi_danger, Length: 1251, dtype: float64
[[  1. 370.]
 [  2. 474.]
 [  3. 339.]
 [  4.  68.]]


### Descision Trees 

In [19]:
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              }


ds = DecisionTreeClassifier()
ds_cv = RandomizedSearchCV(estimator=ds, param_distributions=random_grid, n_iter=100, scoring='f1_weighted')
ds_cv.fit(x_train, y_train)
y_pred_ds = ds_cv.predict(x_test)
print(accuracy_score(y_test, y_pred_ds))
print(cross_val_score(ds, x_train, y_train, cv=3))

0.5143769968051118
[0.56230032 0.41853035 0.56089744]


In [20]:
print(confusion_matrix(y_test, y_pred_ds))

[[63 25  9  1]
 [26 68 25  2]
 [ 7 32 28  6]
 [ 3  5 11  2]]


In [21]:

print(classification_report(y_test, y_pred_ds))

              precision    recall  f1-score   support

         1.0       0.64      0.64      0.64        98
         2.0       0.52      0.56      0.54       121
         3.0       0.38      0.38      0.38        73
         4.0       0.18      0.10      0.12        21

    accuracy                           0.51       313
   macro avg       0.43      0.42      0.42       313
weighted avg       0.50      0.51      0.51       313



### Random Forest

In [22]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


rf = RandomForestClassifier(class_weight = "balanced")
rf_cv = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=5, scoring='f1_weighted')
rf_cv.fit(x_train, y_train)
y_pred_rf = rf_cv.predict(x_test)
print(accuracy_score(y_test, y_pred_rf))
print(cross_val_score(rf, x_train, y_train, cv=3))

0.6293929712460063
[0.65814696 0.65495208 0.63141026]


In [24]:
print(confusion_matrix(y_test, y_pred_rf))

[[73 19  6  0]
 [16 76 28  1]
 [ 3 24 46  0]
 [ 0  1 18  2]]


In [27]:
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

         1.0       0.79      0.74      0.77        98
         2.0       0.63      0.63      0.63       121
         3.0       0.47      0.63      0.54        73
         4.0       0.67      0.10      0.17        21

    accuracy                           0.63       313
   macro avg       0.64      0.52      0.53       313
weighted avg       0.65      0.63      0.62       313



### Extra Forest

In [28]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

et = ExtraTreesClassifier(class_weight = "balanced")
et_cv = RandomizedSearchCV(estimator=et, param_distributions=random_grid, n_iter=5, scoring='f1_weighted')
et_cv.fit(x_train, y_train)
y_pred_et = et_cv.predict(x_test)
print(accuracy_score(y_test, y_pred_et))
print(cross_val_score(rf, x_train, y_train, cv=3))

0.6198083067092651
[0.63578275 0.64217252 0.6474359 ]


In [30]:
print(confusion_matrix(y_test, y_pred_et))

[[75 15  8  0]
 [20 68 31  2]
 [ 3 18 46  6]
 [ 1  1 14  5]]


In [31]:
print(classification_report(y_test, y_pred_et))

              precision    recall  f1-score   support

         1.0       0.76      0.77      0.76        98
         2.0       0.67      0.56      0.61       121
         3.0       0.46      0.63      0.53        73
         4.0       0.38      0.24      0.29        21

    accuracy                           0.62       313
   macro avg       0.57      0.55      0.55       313
weighted avg       0.63      0.62      0.62       313

